In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

endpoint   = os.getenv("MINIO_ENDPOINT")
bucket     = os.getenv("MINIO_BUCKET")
parquet    = os.getenv("MINIO_PARQUET_PATH")
access_key = os.getenv("MINIO_ACCESS_KEY")
secret_key = os.getenv("MINIO_SECRET_KEY")

storage_opts = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {"endpoint_url": endpoint},
}

# Exemplo: abrir o parquet de 2025
path = f"s3://{bucket}/{parquet}/add_producao_2025.parquet"
df = pd.read_parquet(path, storage_options=storage_opts)

In [14]:
# Explorar colunas relacionadas às IES
print("🔍 COLUNAS DISPONÍVEIS:")
print("=" * 50)
columns = df.columns.tolist()
ies_columns = [col for col in columns if any(word in col.lower() for word in ['ies', 'entidade', 'instituicao', 'universidade', 'escola', 'centro', 'faculdade'])]

print("📋 Colunas relacionadas a IES:")
for col in ies_columns:
    print(f"   • {col}")

print(f"\n📊 Total de colunas: {len(columns)}")
print(f"📊 Colunas de IES encontradas: {len(ies_columns)}")

# Vamos ver algumas colunas específicas que podem ser IES
potential_ies = ['nm_entidade_ensino', 'sg_entidade_ensino', 'cd_entidade_ensino', 'nm_ies', 'cd_ies']
existing_ies = [col for col in potential_ies if col in columns]

print(f"\n🎯 Colunas esperadas de IES encontradas:")
for col in existing_ies:
    print(f"   • {col}")

# Mostrar primeiras 5 colunas para entender estrutura
print(f"\n📝 Primeiras 10 colunas do dataset:")
for i, col in enumerate(columns[:10]):
    print(f"   {i+1:2d}. {col}")

🔍 COLUNAS DISPONÍVEIS:
📋 Colunas relacionadas a IES:
   • id_add_foto_programa_ies
   • sg_ies
   • nm_ies
   • cd_entidade_capes
   • cd_ies_emec

📊 Total de colunas: 36
📊 Colunas de IES encontradas: 5

🎯 Colunas esperadas de IES encontradas:
   • nm_ies

📝 Primeiras 10 colunas do dataset:
    1. an_base
    2. id_add_foto_programa
    3. id_add_foto_programa_ies
    4. cd_programa
    5. nm_programa
    6. sg_ies
    7. nm_ies
    8. cd_entidade_capes
    9. cd_ies_emec
   10. id_add_producao_intelectual


In [15]:
# Analisar dados das IES
print("🏛️ ANÁLISE DAS INSTITUIÇÕES DE ENSINO SUPERIOR")
print("=" * 60)

# Verificar informações das IES
ies_cols = ['sg_ies', 'nm_ies', 'cd_entidade_capes', 'cd_ies_emec']
print("📊 ESTATÍSTICAS BÁSICAS:")
for col in ies_cols:
    unique_count = df[col].nunique()
    null_count = df[col].isnull().sum()
    total = len(df)
    print(f"   • {col:20} - Únicos: {unique_count:4d} | Nulos: {null_count:6d} | Total: {total}")

print("\n🔍 AMOSTRA DOS DADOS:")
sample_ies = df[ies_cols].drop_duplicates().head(10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(sample_ies)

print(f"\n📈 TOTAL DE IES ÚNICAS:")
unique_ies = df[['sg_ies', 'nm_ies', 'cd_entidade_capes', 'cd_ies_emec']].drop_duplicates()
print(f"   • Número de IES únicas: {len(unique_ies)}")

# Verificar se há inconsistências
print(f"\n⚠️  VERIFICAÇÃO DE CONSISTÊNCIA:")
print(f"   • IES por sigla: {df['sg_ies'].nunique()}")
print(f"   • IES por nome: {df['nm_ies'].nunique()}")
print(f"   • IES por código CAPES: {df['cd_entidade_capes'].nunique()}")
print(f"   • IES por código EMEC: {df['cd_ies_emec'].nunique()}")

🏛️ ANÁLISE DAS INSTITUIÇÕES DE ENSINO SUPERIOR
📊 ESTATÍSTICAS BÁSICAS:
   • sg_ies               - Únicos:  498 | Nulos:      0 | Total: 1254107
   • nm_ies               - Únicos:  499 | Nulos:      0 | Total: 1254107
   • cd_entidade_capes    - Únicos:  499 | Nulos:      0 | Total: 1254107
   • cd_ies_emec          - Únicos:  364 | Nulos: 105560 | Total: 1254107

🔍 AMOSTRA DOS DADOS:
      sg_ies                                             nm_ies  \
0        UFC                      UNIVERSIDADE FEDERAL DO CEARÁ   
1        UEL                  UNIVERSIDADE ESTADUAL DE LONDRINA   
2       UFRN        UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE   
3       UEPB                   UNIVERSIDADE ESTADUAL DA PARAIBA   
4    FIOCRUZ                    FUNDACAO OSWALDO CRUZ (FIOCRUZ)   
5       UFPE                 UNIVERSIDADE FEDERAL DE PERNAMBUCO   
6        UFU                 UNIVERSIDADE FEDERAL DE UBERLÂNDIA   
7        UPE                         UNIVERSIDADE DE PERNAMBUCO   
8  UNESP

In [16]:
# Verificar algumas IES específicas
print("🎯 EXEMPLOS DE IES:")
print("=" * 40)

# Pegar algumas IES para exemplificar
sample_ies = df[['sg_ies', 'nm_ies', 'cd_entidade_capes', 'cd_ies_emec']].drop_duplicates().head(5)
for idx, row in sample_ies.iterrows():
    print(f"\n📋 IES {idx + 1}:")
    print(f"   • Sigla: {row['sg_ies']}")
    print(f"   • Nome: {row['nm_ies']}")
    print(f"   • Código CAPES: {row['cd_entidade_capes']}")
    print(f"   • Código EMEC: {row['cd_ies_emec']}")

print(f"\n📊 RESUMO GERAL:")
total_producoes = len(df)
total_ies_unicas = len(df[['sg_ies', 'nm_ies', 'cd_entidade_capes', 'cd_ies_emec']].drop_duplicates())
print(f"   • Total de produções: {total_producoes:,}")
print(f"   • Total de IES únicas: {total_ies_unicas}")
print(f"   • Média de produções por IES: {total_producoes / total_ies_unicas:.1f}")

🎯 EXEMPLOS DE IES:

📋 IES 1:
   • Sigla: UFC
   • Nome: UNIVERSIDADE FEDERAL DO CEARÁ
   • Código CAPES: 22001018
   • Código EMEC: 583

📋 IES 2:
   • Sigla: UEL
   • Nome: UNIVERSIDADE ESTADUAL DE LONDRINA
   • Código CAPES: 40002012
   • Código EMEC: 9

📋 IES 3:
   • Sigla: UFRN
   • Nome: UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
   • Código CAPES: 23001011
   • Código EMEC: 570

📋 IES 4:
   • Sigla: UEPB
   • Nome: UNIVERSIDADE ESTADUAL DA PARAIBA
   • Código CAPES: 24004014
   • Código EMEC: 550

📋 IES 5:
   • Sigla: FIOCRUZ
   • Nome: FUNDACAO OSWALDO CRUZ (FIOCRUZ)
   • Código CAPES: 31010016
   • Código EMEC: None

📊 RESUMO GERAL:
   • Total de produções: 1,254,107
   • Total de IES únicas: 500
   • Média de produções por IES: 2508.2


In [17]:
# Verificar quais anos estão disponíveis
print("📅 VERIFICANDO ANOS DISPONÍVEIS:")
print("=" * 40)

available_years = []

# Testar anos de 2013 a 2025
for year in range(2013, 2026):
    try:
        path = f"s3://{bucket}/{parquet}/add_producao_{year}.parquet"
        df_test = pd.read_parquet(path, storage_options=storage_opts)
        available_years.append(year)
        print(f"   ✅ {year}: {len(df_test):,} registros")
    except Exception as e:
        print(f"   ❌ {year}: Não disponível")

print(f"\n📊 ANOS DISPONÍVEIS: {available_years}")
print(f"   • Total de anos: {len(available_years)}")
print(f"   • Período: {min(available_years) if available_years else 'N/A'} - {max(available_years) if available_years else 'N/A'}")

📅 VERIFICANDO ANOS DISPONÍVEIS:
   ✅ 2013: 938,590 registros
   ✅ 2014: 959,715 registros
   ✅ 2015: 967,845 registros
   ✅ 2016: 934,116 registros
   ✅ 2017: 1,133,868 registros
   ✅ 2018: 1,164,348 registros
   ✅ 2019: 1,254,107 registros
   ✅ 2020: 1,143,572 registros
   ✅ 2021: 1,220,848 registros
   ✅ 2022: 1,080,788 registros


KeyboardInterrupt: 

In [11]:
df.columns

Index(['an_base', 'id_add_foto_programa', 'id_add_foto_programa_ies',
       'cd_programa', 'nm_programa', 'sg_ies', 'nm_ies', 'cd_entidade_capes',
       'cd_ies_emec', 'id_add_producao_intelectual', 'id_producao_intelectual',
       'nm_producao', 'id_tipo_producao', 'nm_tipo_producao',
       'id_subtipo_producao', 'nm_subtipo_producao', 'id_formulario_producao',
       'nm_formulario', 'id_add_contexto', 'id_area_concentracao',
       'nm_area_concentracao', 'id_linha_pesquisa', 'nm_linha_pesquisa',
       'id_projeto', 'nm_projeto', 'dh_inicio_area_conc', 'dh_fim_area_conc',
       'dh_inicio_linha', 'dh_fim_linha', 'in_glosa', 'sg_estrato',
       'nm_evento_classificacao', 'cd_identificador_veiculo',
       'ds_titulo_padronizado', 'in_periodico', 'id_valor_lista'],
      dtype='object')

In [10]:
pd.set_option('display.max_columns', None)
df.head

<bound method NDFrame.head of          an_base  id_add_foto_programa  id_add_foto_programa_ies  \
0           2019                105155                    171447   
1           2019                105168                    171439   
2           2019                 83590                    147062   
3           2019                109037                    176040   
4           2019                102694                    168816   
...          ...                   ...                       ...   
1254102     2019                105003                    171253   
1254103     2019                103319                    169502   
1254104     2019                103679                    169897   
1254105     2019                108333                    175197   
1254106     2019                 91957                    156574   

           cd_programa                                   nm_programa   sg_ies  \
0        22001018071P7                 ADMINISTRAÇÃO E CONTROLADORIA    

In [12]:
df.iloc[0]

an_base                                                                     2019
id_add_foto_programa                                                      105155
id_add_foto_programa_ies                                                  171447
cd_programa                                                        22001018071P7
nm_programa                                        ADMINISTRAÇÃO E CONTROLADORIA
sg_ies                                                                       UFC
nm_ies                                             UNIVERSIDADE FEDERAL DO CEARÁ
cd_entidade_capes                                                       22001018
cd_ies_emec                                                                  583
id_add_producao_intelectual                                             29455027
id_producao_intelectual                                                  8268253
nm_producao                    FATORES QUE INFLUENCIAM A UTILIZAÇÃO DE ARTEFA...
id_tipo_producao            

In [13]:
df.iloc[0]['nm_producao']

'FATORES QUE INFLUENCIAM A UTILIZAÇÃO DE ARTEFATOS DE CONTABILIDADE GERENCIAL EM EMPRESAS CEARENSES'